In [ ]:
#import module
import tensorflow as tf
from PIL import Image
import numpy as np
import os
# print(tf.__version__)

In [ ]:
#convert data
ocr_string = """0123456789ABCDEFGHIJKLMNPQRSTUVWXYZ"""
ocr_dict = {}
item=0
for i in ocr_string:
    ocr_dict[i]=item
    item+=1
print(ocr_dict)
size=(32,32)
total_one_char=10
    
def load_data(folder_path):
    x=[];y=[]
    for foldername in os.listdir(folder_path):
        if foldername not in "&#@$":
        # print(foldername)
            folder_path_new=os.path.join(folder_path,foldername)
            cnt=0
            for filename in os.listdir(folder_path_new):
                if cnt>=total_one_char:break
                cnt+=1
                file_path=os.path.join(folder_path_new,filename)
                img=Image.open(file_path)
                img_gray=img.convert('L')
                img_rz=img_gray.resize(size)
                x.append(img_rz)
                y.append(ocr_dict[foldername])
            print(foldername,cnt)
    x_np=np.array(x)
    y_np=np.array(y)
    index_rdn=np.arange(y_np.shape[0])
    np.random.shuffle(index_rdn)
    x_np=x_np[index_rdn]
    y_np=y_np[index_rdn]
    x_np=x_np/255.0
    x_np=np.expand_dims(x_np,axis=-1)
    y_np=np.expand_dims(y_np,axis=-1)
    return x_np,y_np


folder_path='/kaggle/input/handwritten-characters/'
train_folder_path='Train'
test_folder_path='Validation'

x_train,y_train=load_data(folder_path+train_folder_path)
x_test,y_test=load_data(folder_path+test_folder_path)

np.save(('/kaggle/working/x_train.npy'),x_train)
np.save(('/kaggle/working/y_train.npy'),y_train)
np.save('/kaggle/working/y_test.npy',x_test)
np.save('/kaggle/working/y_test.npy',y_test)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)



In [ ]:
#load data
path='/kaggle/working/'
x_train=np.load(path+'x_train.npy')
y_train=np.load(path+'y_train.npy')
x_test=np.load(path+'x_test.npy')
y_test=np.load(path+'y_test.npy')
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)


In [ ]:
#build model
model=tf.keras.Sequential([
tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',input_shape=(32,32,1)),
tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu',padding='same'),
tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu',padding='same'),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(256,activation='relu'),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(35,activation='softmax')
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()


In [ ]:
#train model and save model
model.fit(x_train,y_train,epochs=10,batch_size=50)
model.save(path+'model.h5')

In [ ]:
#load_model
model = tf.keras.models.load_model(path+'model.h5')
model.summary()

In [ ]:
#test model
model.evaluate(x_test,y_test,batch_size=10)

In [ ]:
#test thực tế
for i in range(1,17):
    img=Image.open('tst/'+str(i)+'.jpg')
    img_gray=img.convert("L")
    img_resize=img_gray.resize((32,32))
    img_array=np.array(img_resize)
    img_array=img_array/255.0
    img_array=np.expand_dims(img_array,axis=0)
    img_array=np.expand_dims(img_array,axis=-1)
    predictions=model.predict(img_array)
    predicted_class = tf.argmax(predictions, axis=-1)
    print(i,predicted_class.numpy()) # In ra index của lớp được dự đoán

